In [ ]:
"""
Created: 6/7

Purpose: To incorporate the synapses
into the neuron object

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-06-14 20:30:29,286 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-14 20:30:29,288 - settings - Setting database.user to celiib
INFO - 2021-06-14 20:30:29,289 - settings - Setting database.password to newceliipass
INFO - 2021-06-14 20:30:29,289 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-14 20:30:29,290 - settings - Setting database.user to celiib
INFO - 2021-06-14 20:30:29,291 - settings - Setting database.password to newceliipass
INFO - 2021-06-14 20:30:29,296 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/de

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-06-14 20:30:29,850 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-14 20:30:29,901 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-14 20:30:29,902 - settings - Setting database.user to celiib
INFO - 2021-06-14 20:30:29,903 - settings - Setting database.password to newceliipass
INFO - 2021-06-14 20:30:29,904 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-14 20:30:30,523 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-14 20:30:30,605 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-14 20:30:30,607 - settings - Setting database.user to celiib
INFO - 2021-06-14 20:30:30,608 - settings - Setting database.password to newceliipass
INFO - 2021-06-14 20:30:30,609 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-14 20:30:30,610 - settings - Setting database.user to celiib
INFO - 2021-06-14 20:30:30,611 - settings - Setting database.pa

In [3]:
#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-06-14 20:30:31,461 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-14 20:30:31,463 - settings - Setting database.user to celiib
INFO - 2021-06-14 20:30:31,464 - settings - Setting database.password to newceliipass
INFO - 2021-06-14 20:30:31,465 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-14 20:30:32,236 - settings - Setting enable_python_native_blobs to True


In [4]:
import neuron_utils as nru
import neuron_visualizations as nviz

# Downloading a Neuron to Play with

In [9]:
segment_id = 864691136194042326

In [10]:
neuron_obj  = du.decomposition_with_spine_recalculation(segment_id,split_index = 0)

**Using table __decomposition_axon for table_to_neuron_objs**
Decompressing Neuron in minimal output mode...please wait
Dont need spine recalculation
Number of Neurons found = 1


In [ ]:
# nviz.visualize_neuron(neuron_obj,
#                      limb_branch_dict="all")

# Pulling Down the Synapses Before the Proofreading

In [ ]:
#original_mesh = du.fetch_segment_id_mesh(neuron_obj.segment_id)

In [ ]:
nviz.plot_objects(neuron_obj.mesh)

In [ ]:
minnie.AutoProofreadValidationSegmentMap4() & dict(segment_id=segment_id)

In [ ]:
nviz.visualize_neuron(neuron_obj_syn,
                     limb_branch_dict="all")

In [ ]:
import synapse_utils as syu
neuron_obj_syn = syu.add_synapses_to_neuron_obj(neuron_obj,
                                                segment_id = segment_id,
                            validation = True,
                            verbose  = True,
                            #original_mesh = original_mesh,
                            plot_valid_error_synapses = False,
                            calculate_synapse_soma_distance = False,
                            add_valid_synapses = True,
                              add_error_synapses=False)

In [ ]:
neuron_obj.synapses

# Filtering the Neuron and then adding back the errors

In [ ]:
import proofreading_utils as pru
o_neuron, filtering_info = pru.apply_proofreading_filters_to_neuron(input_neuron = neuron_obj,
                                        filter_list = pru.v5_exc_filters(),
                                        plot_limb_branch_filter_with_disconnect_effect=False,
                                        plot_limb_branch_filter_away=False,
                                        plot_final_neuron=False,

                                        return_error_info=True,
                                         verbose=True,
                                        verbose_outline=True,
                                        return_limb_branch_dict_to_cancel = True)
nviz.visualize_neuron(o_neuron,limb_branch_dict="all")

In [ ]:
import neuron
o_neuron_cp = neuron.Neuron(o_neuron)
syu.add_synapses_to_neuron_obj(o_neuron_cp,
                            validation = True,
                            verbose  = True,
                            original_mesh = original_mesh,
                            plot_valid_error_synapses = False,
                            calculate_synapse_soma_distance = False,
                            add_valid_synapses = False,
                              add_error_synapses=True)
o_neuron_cp.n_synapses_total

# How to Query Synapses

In [ ]:
syu.synapses_df(neuron_obj)

ret_df = syu.query_synapses(neuron_obj,
query = "label == 'soma'",
verbose = True,
return_index = True)

ret_df